In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

from functools import partial
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scipy.ndimage.interpolation import shift

F:\ML\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
MNIST_PATH = os.path.join("datasets", "MNIST")
FILE_NAME = "train.csv"

In [3]:
def load_mnist_data(mnist_path=MNIST_PATH, file_name = FILE_NAME):
    csv_path = os.path.join(mnist_path, file_name)
    return pd.read_csv(csv_path)

In [4]:
def random_batch(X_train, y_train, batch_size):
    rnd_index = np.random.randint(0, len(X_train), batch_size)
    X_batch = X_train[rnd_index, :]
    y_batch = y_train[rnd_index]
    return X_batch, y_batch

In [5]:
def shift_image(image, dx, dy):
    image = image.reshape((28, 28))
    shifted_image = shift(image, [dy, dx], cval=0, mode="constant")
    return shifted_image.reshape([-1])

In [6]:
mnist = load_mnist_data()

In [7]:
X_train, X_val, y_train, y_val = train_test_split(mnist.values[:, 1:], mnist.values[:, 0], test_size = 0.2, random_state = 42)

In [8]:
X_train_augmented = [image for image in X_train]
y_train_augmented = [label for label in y_train]

for dx, dy in ((1, 0), (-1, 0), (0, 1), (0, -1)):
    for image, label in zip(X_train, y_train):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)
        
for dx, dy in ((2, 0), (-2, 0), (0, 2), (0, -2)):
    for image, label in zip(X_train, y_train):
        X_train_augmented.append(shift_image(image, dx, dy))
        y_train_augmented.append(label)
        
X_train_augmented = np.array(X_train_augmented)
y_train_augmented = np.array(y_train_augmented)

In [9]:
shuffle_idx  = np.random.permutation(len(X_train_augmented))
X_train_augmented = X_train_augmented[shuffle_idx]
y_train_augmented = y_train_augmented[shuffle_idx]

In [10]:
scaler = MinMaxScaler()
X_train_augmented = scaler.fit_transform(X_train_augmented)
X_val = scaler.fit_transform(X_val)

F:\ML\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [11]:
tf.reset_default_graph()

In [12]:
n_inputs = 28*28
n_hidden1 = 540
n_hidden2 = 260
n_hidden3 = 140
n_hidden4 = 70
n_outputs = 10

In [13]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")
training = tf.placeholder_with_default(False, shape=(), name="training")

In [14]:
my_batch_norm_layer = partial(tf.layers.batch_normalization, training = training, momentum = 0.9)
he_init = tf.contrib.layers.variance_scaling_initializer()

dropout_rate = 0.5

X_drop = tf.layers.dropout(X, 0.6, training=training)

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X_drop, n_hidden1, name="hidden1", activation=tf.nn.elu, kernel_initializer = he_init)
    hidden1_drop = tf.layers.dropout(hidden1, dropout_rate, training=training)
    bn1 = my_batch_norm_layer(hidden1_drop)
    bn1_act = tf.nn.elu(bn1)
    
    hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2", activation=tf.nn.elu, kernel_initializer = he_init)
    hidden2_drop = tf.layers.dropout(hidden2, dropout_rate, training=training)
    bn2 = my_batch_norm_layer(hidden2_drop)
    bn2_act = tf.nn.elu(bn2)
    
    hidden3 = tf.layers.dense(bn2_act, n_hidden3, name="hidden3", activation=tf.nn.elu, kernel_initializer = he_init)
    hidden3_drop = tf.layers.dropout(hidden3, 0.4, training=training)
    bn3 = my_batch_norm_layer(hidden3_drop)
    bn3_act = tf.nn.elu(bn3)
    
    hidden4 = tf.layers.dense(bn3_act, n_hidden4, name="hidden4", activation=tf.nn.elu, kernel_initializer = he_init)
    hidden4_drop = tf.layers.dropout(hidden4, 0.3, training=training)
    bn4 = my_batch_norm_layer(hidden4_drop)
    bn4_act = tf.nn.elu(bn4)
    
#    hidden5 = tf.layers.dense(bn4_act, n_hidden5, name="hidden5", activation=tf.nn.elu, kernel_initializer = he_init)
#    bn5 = my_batch_norm_layer(hidden5)
#    bn5_act = tf.nn.elu(bn5)
        
#    hidden6 = tf.layers.dense(bn5_act, n_hidden6, name="hidden6", activation=tf.nn.elu, kernel_initializer = he_init)
#    bn6 = my_batch_norm_layer(hidden6)
#    bn6_act = tf.nn.elu(bn6)
        
#    hidden7 = tf.layers.dense(bn6_act, n_hidden7, name="hidden7", activation=tf.nn.elu, kernel_initializer = he_init)
#    bn7 = my_batch_norm_layer(hidden7)
 #   bn7_act = tf.nn.elu(bn4)
    
    outputs = tf.layers.dense(bn4_act, n_outputs, name="outputs")

In [15]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=outputs) 
    loss = tf.reduce_mean(xentropy, name="loss")

In [16]:
learning_rate = 0.01
momentum = 0.9
with tf.name_scope("train"):
    optimizer = tf.train.MomentumOptimizer(learning_rate = learning_rate, momentum = momentum, use_nesterov=True)
    training_op = optimizer.minimize(loss)

In [17]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(outputs, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [18]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
n_epochs = 10000
batch_size = 100
n_batches = int(np.ceil(X_train_augmented.shape[0]/ batch_size))

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

best_accuracy = 0

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(n_batches):
            X_batch, y_batch = random_batch(X_train_augmented, y_train_augmented, batch_size)
            sess.run([training_op, extra_update_ops], feed_dict={training: True, X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_val, y: y_val})
        print(epoch, "Правильность при обучении:", acc_train,
                     "Правильность при проверке:", acc_test)
        
        if acc_test > best_accuracy:
            best_accuracy = acc_test
            save_path = saver.save(sess, "./mnist/best_model_final4/best_mnist_model_final.ckpt")
            print("Найдена лучшая модель на ", epoch+1, " итерации с правильностью при проверке:", acc_test)

0 Ошибка при обучении: 0.9 Ошибка при проверке: 0.9166667
Найдена лучшая модель на  1  итерации с точностью при проверке: 0.9166667
1 Ошибка при обучении: 0.91 Ошибка при проверке: 0.9305952
Найдена лучшая модель на  2  итерации с точностью при проверке: 0.9305952
2 Ошибка при обучении: 0.94 Ошибка при проверке: 0.9266667
3 Ошибка при обучении: 0.94 Ошибка при проверке: 0.9286905
4 Ошибка при обучении: 0.84 Ошибка при проверке: 0.9223809
5 Ошибка при обучении: 0.96 Ошибка при проверке: 0.9354762
Найдена лучшая модель на  6  итерации с точностью при проверке: 0.9354762
6 Ошибка при обучении: 0.93 Ошибка при проверке: 0.93988097
Найдена лучшая модель на  7  итерации с точностью при проверке: 0.93988097
7 Ошибка при обучении: 0.93 Ошибка при проверке: 0.9370238
8 Ошибка при обучении: 0.93 Ошибка при проверке: 0.9402381
Найдена лучшая модель на  9  итерации с точностью при проверке: 0.9402381
9 Ошибка при обучении: 0.93 Ошибка при проверке: 0.9413095
Найдена лучшая модель на  10  итерации 

100 Ошибка при обучении: 0.96 Ошибка при проверке: 0.9754762
Найдена лучшая модель на  101  итерации с точностью при проверке: 0.9754762
101 Ошибка при обучении: 0.96 Ошибка при проверке: 0.97440475
102 Ошибка при обучении: 0.97 Ошибка при проверке: 0.9752381
103 Ошибка при обучении: 0.99 Ошибка при проверке: 0.97571427
Найдена лучшая модель на  104  итерации с точностью при проверке: 0.97571427
104 Ошибка при обучении: 0.98 Ошибка при проверке: 0.9753571
105 Ошибка при обучении: 0.97 Ошибка при проверке: 0.97440475
106 Ошибка при обучении: 0.96 Ошибка при проверке: 0.97369045
107 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9767857
Найдена лучшая модель на  108  итерации с точностью при проверке: 0.9767857
108 Ошибка при обучении: 0.95 Ошибка при проверке: 0.9745238
109 Ошибка при обучении: 0.98 Ошибка при проверке: 0.9746429
110 Ошибка при обучении: 0.97 Ошибка при проверке: 0.9745238
111 Ошибка при обучении: 0.95 Ошибка при проверке: 0.9740476
112 Ошибка при обучении: 1.0 Ошибка

222 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9790476
223 Ошибка при обучении: 0.98 Ошибка при проверке: 0.9791667
224 Ошибка при обучении: 0.98 Ошибка при проверке: 0.9790476
225 Ошибка при обучении: 0.95 Ошибка при проверке: 0.9766667
226 Ошибка при обучении: 0.98 Ошибка при проверке: 0.97833335
227 Ошибка при обучении: 0.98 Ошибка при проверке: 0.9791667
228 Ошибка при обучении: 0.99 Ошибка при проверке: 0.97964287
229 Ошибка при обучении: 0.97 Ошибка при проверке: 0.9785714
230 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9779762
231 Ошибка при обучении: 0.97 Ошибка при проверке: 0.9791667
232 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9777381
233 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9775
234 Ошибка при обучении: 0.99 Ошибка при проверке: 0.97809523
235 Ошибка при обучении: 0.98 Ошибка при проверке: 0.97809523
236 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9790476
237 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9786905
238 Ошибка при обучении

356 Ошибка при обучении: 1.0 Ошибка при проверке: 0.97952384
357 Ошибка при обучении: 0.97 Ошибка при проверке: 0.98083335
358 Ошибка при обучении: 0.96 Ошибка при проверке: 0.98035717
359 Ошибка при обучении: 0.98 Ошибка при проверке: 0.97964287
360 Ошибка при обучении: 0.97 Ошибка при проверке: 0.9798809
361 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9792857
362 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9786905
363 Ошибка при обучении: 0.98 Ошибка при проверке: 0.9785714
364 Ошибка при обучении: 0.99 Ошибка при проверке: 0.97940475
365 Ошибка при обучении: 0.97 Ошибка при проверке: 0.97964287
366 Ошибка при обучении: 0.97 Ошибка при проверке: 0.9797619
367 Ошибка при обучении: 0.98 Ошибка при проверке: 0.9790476
368 Ошибка при обучении: 0.99 Ошибка при проверке: 0.98142856
Найдена лучшая модель на  369  итерации с точностью при проверке: 0.98142856
369 Ошибка при обучении: 0.99 Ошибка при проверке: 0.98083335
370 Ошибка при обучении: 0.98 Ошибка при проверке: 0.97892857


488 Ошибка при обучении: 0.99 Ошибка при проверке: 0.98035717
489 Ошибка при обучении: 0.99 Ошибка при проверке: 0.98011905
490 Ошибка при обучении: 0.96 Ошибка при проверке: 0.9811905
491 Ошибка при обучении: 0.96 Ошибка при проверке: 0.97892857
492 Ошибка при обучении: 1.0 Ошибка при проверке: 0.9802381
493 Ошибка при обучении: 0.98 Ошибка при проверке: 0.9798809
494 Ошибка при обучении: 0.98 Ошибка при проверке: 0.9802381
495 Ошибка при обучении: 0.98 Ошибка при проверке: 0.9778572
496 Ошибка при обучении: 0.98 Ошибка при проверке: 0.98059523
497 Ошибка при обучении: 1.0 Ошибка при проверке: 0.9802381
498 Ошибка при обучении: 1.0 Ошибка при проверке: 0.98071426
499 Ошибка при обучении: 0.98 Ошибка при проверке: 0.9790476
500 Ошибка при обучении: 0.98 Ошибка при проверке: 0.98130953
501 Ошибка при обучении: 1.0 Ошибка при проверке: 0.97952384
502 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9792857
503 Ошибка при обучении: 0.96 Ошибка при проверке: 0.98142856
504 Ошибка при обуче

...

1021 Ошибка при обучении: 0.96 Ошибка при проверке: 0.9817857
1022 Ошибка при обучении: 0.94 Ошибка при проверке: 0.97940475
1023 Ошибка при обучении: 1.0 Ошибка при проверке: 0.9811905
1024 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9804762
1025 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9825
Найдена лучшая модель на  1026  итерации с точностью при проверке: 0.9825
1026 Ошибка при обучении: 0.99 Ошибка при проверке: 0.97964287
1027 Ошибка при обучении: 0.96 Ошибка при проверке: 0.9816667
1028 Ошибка при обучении: 1.0 Ошибка при проверке: 0.98083335
1029 Ошибка при обучении: 0.98 Ошибка при проверке: 0.98083335
1030 Ошибка при обучении: 0.98 Ошибка при проверке: 0.98142856
1031 Ошибка при обучении: 1.0 Ошибка при проверке: 0.9823809
1032 Ошибка при обучении: 1.0 Ошибка при проверке: 0.9804762
1033 Ошибка при обучении: 1.0 Ошибка при проверке: 0.98059523
1034 Ошибка при обучении: 0.98 Ошибка при проверке: 0.97761905
1035 Ошибка при обучении: 0.99 Ошибка при проверке: 0.98059

1152 Ошибка при обучении: 0.98 Ошибка при проверке: 0.98130953
1153 Ошибка при обучении: 0.95 Ошибка при проверке: 0.98083335
1154 Ошибка при обучении: 0.99 Ошибка при проверке: 0.98083335
1155 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9804762
1156 Ошибка при обучении: 0.97 Ошибка при проверке: 0.98083335
1157 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9804762
1158 Ошибка при обучении: 0.96 Ошибка при проверке: 0.9811905
1159 Ошибка при обучении: 1.0 Ошибка при проверке: 0.98071426
1160 Ошибка при обучении: 0.99 Ошибка при проверке: 0.98142856
1161 Ошибка при обучении: 0.98 Ошибка при проверке: 0.9804762
1162 Ошибка при обучении: 0.98 Ошибка при проверке: 0.98035717
1163 Ошибка при обучении: 0.99 Ошибка при проверке: 0.97892857
1164 Ошибка при обучении: 0.99 Ошибка при проверке: 0.98011905
1165 Ошибка при обучении: 0.99 Ошибка при проверке: 0.98071426
1166 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9816667
1167 Ошибка при обучении: 0.96 Ошибка при проверке: 0.9804762

1283 Ошибка при обучении: 1.0 Ошибка при проверке: 0.9815476
1284 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9809524
1285 Ошибка при обучении: 0.98 Ошибка при проверке: 0.98083335
1286 Ошибка при обучении: 0.97 Ошибка при проверке: 0.9816667
1287 Ошибка при обучении: 1.0 Ошибка при проверке: 0.9810714
1288 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9809524
1289 Ошибка при обучении: 0.99 Ошибка при проверке: 0.98214287
1290 Ошибка при обучении: 0.96 Ошибка при проверке: 0.9797619
1291 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9816667
1292 Ошибка при обучении: 0.99 Ошибка при проверке: 0.97952384
1293 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9815476
1294 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9817857
1295 Ошибка при обучении: 0.97 Ошибка при проверке: 0.9802381
1296 Ошибка при обучении: 0.98 Ошибка при проверке: 0.9810714
1297 Ошибка при обучении: 0.97 Ошибка при проверке: 0.97952384
1298 Ошибка при обучении: 0.98 Ошибка при проверке: 0.98083335
1299 

1415 Ошибка при обучении: 0.98 Ошибка при проверке: 0.98142856
1416 Ошибка при обучении: 0.96 Ошибка при проверке: 0.9809524
1417 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9804762
1418 Ошибка при обучении: 0.98 Ошибка при проверке: 0.98
1419 Ошибка при обучении: 0.97 Ошибка при проверке: 0.9809524
1420 Ошибка при обучении: 1.0 Ошибка при проверке: 0.98142856
1421 Ошибка при обучении: 0.98 Ошибка при проверке: 0.98214287
1422 Ошибка при обучении: 0.98 Ошибка при проверке: 0.98035717
1423 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9809524
1424 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9802381
1425 Ошибка при обучении: 0.97 Ошибка при проверке: 0.98083335
1426 Ошибка при обучении: 1.0 Ошибка при проверке: 0.98011905
1427 Ошибка при обучении: 0.99 Ошибка при проверке: 0.98035717
1428 Ошибка при обучении: 0.96 Ошибка при проверке: 0.9811905
1429 Ошибка при обучении: 0.99 Ошибка при проверке: 0.98261905
1430 Ошибка при обучении: 0.99 Ошибка при проверке: 0.98071426
1431 О

1548 Ошибка при обучении: 0.98 Ошибка при проверке: 0.97952384
1549 Ошибка при обучении: 0.99 Ошибка при проверке: 0.98011905
1550 Ошибка при обучении: 0.98 Ошибка при проверке: 0.98214287
1551 Ошибка при обучении: 0.98 Ошибка при проверке: 0.98011905
1552 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9816667
1553 Ошибка при обучении: 0.99 Ошибка при проверке: 0.98011905
1554 Ошибка при обучении: 0.93 Ошибка при проверке: 0.98011905
1555 Ошибка при обучении: 0.95 Ошибка при проверке: 0.9802381
1556 Ошибка при обучении: 0.98 Ошибка при проверке: 0.98083335
1557 Ошибка при обучении: 0.98 Ошибка при проверке: 0.98083335
1558 Ошибка при обучении: 1.0 Ошибка при проверке: 0.9809524
1559 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9809524
1560 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9817857
1561 Ошибка при обучении: 0.98 Ошибка при проверке: 0.98011905
1562 Ошибка при обучении: 0.99 Ошибка при проверке: 0.9810714
1563 Ошибка при обучении: 1.0 Ошибка при проверке: 0.98083335


In [ ]:
MNIST_PATH = os.path.join("datasets", "MNIST")
FILE_NAME = "test.csv"
csv_path = os.path.join(MNIST_PATH, FILE_NAME)

mnist_test = pd.read_csv(csv_path)

In [ ]:
X_test = scaler.fit_transform(mnist_test.values)

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./mnist/best_model_final2/best_mnist_model_final.ckpt")
    Z = outputs.eval(feed_dict={X: X_test})
    y_pred = np.argmax(Z, axis=1)

In [ ]:
TAXIFARE_PATH = os.path.join("datasets", "MNIST")
csv_path = os.path.join(TAXIFARE_PATH, 'submission2.csv')

In [ ]:
submission = pd.DataFrame(
    {'ImageId': list(range(1, len(y_pred)+1)), 'Label': y_pred},
    columns = ["ImageId", "Label"])
submission.to_csv(csv_path, index = False)